# Thanks Usage

During the 2022 Wishathon, access to the Thanks functionality was increased. We now want to know if this also resulted in increased usage of Thanks. The phab task for this work is [T332441](https://phabricator.wikimedia.org/T332441)

The plan is to grab data on the usage of Thanks for December 2022, and January & February 2023. Thanks is logged in the `logging` table, for which we have monthly snapshots. It might be easiest to grab edit data from MediaWiki history because that's easier than combining the `revision` and `archive` tables.

## NOTES

* Ideally, we'd require thanks to be sent within some specific amount of time after the edit was made, rather than make it open-ended. In this case we'll ignore that because we want the process to be easy.
* Since working with three months of English Wikipedia edit data doesn't fit in memory, meaning I chose to use a temp table on Spark and query MediaWiki history for the edits, we can be slightly more sophisticated and 1) easily ignore bot edits, and 2) focus on "content namespaces".

In [1]:
import json
import datetime as dt

from collections import defaultdict

import numpy as np
import pandas as pd

from wmfdata import hive, spark, mariadb

## Configuration Variables

In [2]:
data_start_ts = dt.datetime(2022, 12, 1, 0, 0, 0)
data_end_ts = dt.datetime(2023, 3, 1, 0, 0 , 0)

wikis = ['enwiki', 'frwiki', 'ptwiki', 'dewiki', 'eswiki', 'bnwiki', 'cswiki', 'arwiki']

In [ ]:
## Modified from the example on https://www.mediawiki.org/w/index.php?title=Extension:Thanks&oldid=3850817
## log_title points to the user name of the user who was thanked. we'll have to join that with user
## to get the user id. We also add `log_type = "thanks"` to use the correct index and speed things up.
## We're going to make the following assumption: there's a limited number of thanks
## sent during the experiment. Hence, we can join "all thanks" with user information for our
## user group without running into performance issues. This also kind of assumes that MariaDB
## caches the sub-query after the first run so it's fast for consecutive queries.

## Notes: log_user_text is no longer available, log_actor reflects who gave the thanks
## log_title is still the user who received the thanks
## The revision ID is stored in log_search, with ls_field set to "thankid" and ls_value set to "rev-{revid}"

## log_search is _not_ sqooped together with the other tables, darn!
## we'll have to do this on a per-wiki basis, then.

thanks_query = '''
SELECT DATABASE() AS wiki_db, user_id, count(*) AS num_thanks
FROM user
JOIN (
    SELECT log_timestamp AS thank_timestamp,
           replace(log_title, '_', ' ') AS receiver
    FROM logging
    WHERE log_type = "thanks"
    AND log_action = "thank"
    AND log_timestamp >= "{start_ts}"
) AS lt
ON user_name = receiver
WHERE user_id IN ({id_list})
AND thank_timestamp < DATE_FORMAT(
                             DATE_ADD(STR_TO_DATE(user_registration, "%Y%m%d%H%i%S"),
                                      INTERVAL 15 DAY),
                         "%Y%m%d%H%i%S")
GROUP BY wiki_db, user_id
'''

The query below won't finish in a reasonable amount of time, probably because we're creating some temporary tables and MariaDB is having a hard time joining them. Based on the results from `EXPLAIN` it'll use `filesort`, which even on a small-ish wiki like `nowiki` won't finish quickly.

I think we'll instead grab edits and thanks separately and let Pandas join them.

In [ ]:
thanks_query = '''
WITH thanks_sent AS (
    SELECT
        log_timestamp,
        CAST(REGEXP_REPLACE(ls_value, "^rev-", "") AS INT) AS rev_id
    FROM logging
    JOIN log_search
    ON log_id = ls_log_id
    WHERE log_type = "thanks"
    AND log_action = "thank"
    AND log_timestamp >= "{start_ts}"
    AND log_timestamp < "{end_ts}"
    AND ls_field = "thankid"
),
edits_made AS (
    SELECT
        rev_id,
        rev_timestamp
    FROM revision
    WHERE rev_timestamp >= "{start_ts}"
    AND rev_timestamp < "{end_ts}"
    UNION
    SELECT
        ar_rev_id,
        ar_timestamp
    FROM archive
    WHERE ar_timestamp >= "{start_ts}"
    AND ar_timestamp < "{end_ts}"
)
SELECT
    substring(rev_timestamp, 1, 8) AS rev_day,
    count(1) AS num_edits,
    count(thanks_sent.rev_id) AS num_thanks
FROM edits_made
LEFT JOIN thanks_sent
ON edits_made.rev_id = thanks_sent.rev_id
GROUP BY substring(rev_timestamp, 1, 8)
'''

## Grabbing Data

In [3]:
thanks_query = '''
WITH thanks_sent AS (
    SELECT
        DATABASE() AS wiki_db,
        CAST(REGEXP_REPLACE(ls_value, "^rev-", "") AS INT) AS rev_id,
        log_timestamp
    FROM logging
    JOIN log_search
    ON log_id = ls_log_id
    WHERE log_type = "thanks"
    AND log_action = "thank"
    AND log_timestamp >= "{start_ts}"
    AND log_timestamp < "{end_ts}"
    AND ls_field = "thankid"
)
SELECT
    *
FROM thanks_sent
'''

In [ ]:
thanks_data = mariadb.run(
    thanks_query.format(
        start_ts = data_start_ts.strftime('%Y%m%d%H%M%S'),
        end_ts = data_end_ts.strftime('%Y%m%d%H%M%S')
    ),
    wikis)

In [7]:
thanks_data.groupby('wiki_db').count()

,rev_id,log_timestamp
wiki_db,,
arwiki,5139,5139
bnwiki,722,722
cswiki,4533,4533
dewiki,33304,33304
enwiki,95478,95478
eswiki,11479,11479
frwiki,21184,21184
ptwiki,4540,4540


## Temp Table on the Cluster

In [32]:
spark_session = spark.create_session()
thanks_sdf = spark_session.createDataFrame(thanks_data)
thanks_sdf.createOrReplaceGlobalTempView('thanks_spike')

23/03/21 14:30:32 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/21 14:30:32 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/03/21 14:30:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/21 14:30:38 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 13000. Attempting port 13001.
23/03/21 14:30:38 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


### Notes

* I removed the `AND page_namespace_is_content` expression to see if we get significant changes. The visualizations show an interesting increase in Thanks usage on `arwiki`.
* I also changed it so that we have three categories of user tenure, as measured by time between user registration and when the edit was made. For simplicity, I set this to "Less than 30 days" (e.g. a "newcomer"), "Less than 1 year", and "More than 1 year". Finer distinctions might be useful, but I also wanted to only have three categories because I'd like to keep the graph similar to the one with three months in it.

In [38]:
combined_query = '''
WITH edits AS (
    SELECT
        mh.wiki_db,
        CASE
            WHEN unix_timestamp(event_timestamp, "yyyy-MM-dd HH:mm:ss.S") -
                    unix_timestamp(event_user_creation_timestamp, "yyyy-MM-dd HH:mm:ss.S") < 86400*30
            THEN "Less than 30 days"
            WHEN unix_timestamp(event_timestamp, "yyyy-MM-dd HH:mm:ss.S") -
                    unix_timestamp(event_user_creation_timestamp, "yyyy-MM-dd HH:mm:ss.S")
                    BETWEEN 86400*30 AND 86400*365
            THEN "Less than 1 year"
            ELSE "More than 1 year"
        END AS user_tenure,
        revision_id AS rev_id,
        TO_DATE(event_timestamp) AS rev_day,
        IF(ts.log_timestamp IS NOT NULL, 1, 0) AS was_thanked
    FROM wmf.mediawiki_history AS mh
    LEFT JOIN global_temp.thanks_spike AS ts
    ON mh.wiki_db = ts.wiki_db
    AND mh.revision_id = ts.rev_id
    WHERE snapshot = "2023-02"
    AND mh.wiki_db IN ({wiki_list})
    AND event_timestamp >= "{start_ts}"
    AND event_timestamp < "{end_ts}"
    AND event_entity = "revision"
    AND event_type = "create"
    AND size(event_user_is_bot_by) = 0
    AND size(event_user_is_bot_by_historical) = 0
    AND event_user_is_anonymous = false 
    AND NOT (revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48)
)
SELECT
    wiki_db,
    rev_day,
    user_tenure,
    count(1) AS num_edits,
    SUM(was_thanked) AS num_thanks
FROM edits
GROUP BY wiki_db, rev_day, user_tenure
'''

In [39]:
combined_data = spark.run(combined_query.format(
    wiki_list = ','.join(['"{}"'.format(w) for w in wikis]),
    start_ts = data_start_ts.strftime('%Y-%m-%d %H:%M:%S'),
    end_ts = data_end_ts.strftime('%Y-%m-%d %H:%M:%S')
))

23/03/21 14:43:21 WARN TaskSetManager: Lost task 2316.0 in stage 1.0 (TID 992) (an-worker1105.eqiad.wmnet executor 35): TaskKilled (Stage cancelled)
23/03/21 14:43:37 WARN TaskSetManager: Stage 5 contains a task of very large size (2842 KiB). The maximum recommended task size is 1000 KiB.


In [ ]:
combined_data.head()

In [41]:
combined_data.to_csv('datasets/thanks-data-3months-by-tenure.tsv',
                     sep = '\t', header = True, index = False)

## Thanks from Different Places

Thanks are not just sent for edits. From a quick check, it looks like the first part of `ls_value` for thanks are the type of event that was thanked. Let's look at that for Norwegian Wikipedia.

In [53]:
thanks_query = r'''
WITH thanks_sent AS (
    SELECT
        DATABASE() AS wiki_db,
        log_timestamp,
        regexp_replace(ls_value, '(\\w+)-.*', '\\1') AS thanks_type
    FROM logging
    JOIN log_search
    ON log_id = ls_log_id
    WHERE log_type = "thanks"
    AND log_action = "thank"
    AND log_timestamp >= "{start_ts}"
    AND log_timestamp < "{end_ts}"
    AND ls_field = "thankid"
)
SELECT
    wiki_db,
    thanks_type,
    count(1) AS num_thanks
FROM thanks_sent
GROUP BY wiki_db, thanks_type
'''

In [ ]:
thanks_types = mariadb.run(
    thanks_query.format(
        start_ts = data_start_ts.strftime('%Y%m%d%H%M%S'),
        end_ts = data_end_ts.strftime('%Y%m%d%H%M%S')
    ),
    wikis)

In [55]:
thanks_types

,wiki_db,thanks_type,num_thanks
0,enwiki,log,839
1,enwiki,rev,94639
2,frwiki,flow,741
3,frwiki,log,328
4,frwiki,rev,20115
5,ptwiki,flow,15
6,ptwiki,log,96
7,ptwiki,rev,4429
8,dewiki,log,692
9,dewiki,rev,32612
